In [1]:
#revisa si el vector es cero
#vector: lista de enteros
def zero_vector(vector):
    if type(vector) is Integer or type(vector) is int:
        return vector == 0
    else:
        for i in range(0,len(vector)):
            if vector[i] != 0:
                return False
    return True

#revisa si la matriz es cero
#matrix: lista de listas de enteros
def zero_matrix(matrix):
    for i in matrix:
        if not zero_vector(i):
            return False
    return True

#imprime la matriz de una forma mas clara
#matriz: lista de listas
def pm(matriz):
    for i in matriz:
        print(i)
        
#transpone la matriz
#matriz: lista de listas
def transpose_matrix(matriz):
    zipped_rows = zip(*matriz)
    return [list(row) for row in zipped_rows]

#soporte de un vector
#vector: lista de enteros
def vector_support(vector):
    indexes = []
    for i in range(len(vector)):
        if vector[i] != 0:
            indexes.append(i)
    return indexes

#genera todos los vectores binarios posibles de longitud n
def generate_binary(n):
    bin_arr = range(0, int(math.pow(2,n)))
    bin_arr = [bin(i)[2:] for i in bin_arr]
    max_len = len(max(bin_arr, key=len))
    bin_arr = [list(i.zfill(max_len)) for i in bin_arr]
    for a in bin_arr:
        for i in range(len(a)):
            a[i] = int(a[i])
    return bin_arr

#calcula las duplas que hacen parte de la q-orbita
#q: entero, numero primo de la q-orbita
#a: dupla de enteros, los enteros a0 y a1 de la q-orbita que se quiere calcular
#r: dupla de enteros, r1 y r2 en I = Zr1 x Zr2
def q_orbits(q,a,r):
    parejas = []
    for i in range(r[0]*r[1]):
        coord1 = (a[0] * q**i) % r[0]
        coord2 = (a[1] * q**i) % r[1]
        if [coord1,coord2] not in parejas:
            parejas.append([coord1,coord2])
    return parejas


def generate_matrix(q,a,r):
    coordinates = []
    for j in a:
        qorbits = q_orbits(q,j,r)
        for o in qorbits:
            if o not in coordinates:
                coordinates.append(o) 
    matrix = [[1 for x in range(r[1])] for y in range(r[0])]
    for i,j in coordinates:
         matrix[i][j] = 0
    return matrix

    #calcula el omega de una hipercolumna en una matriz
#matriz: lista de listas de enteros
#vector: entero, el indice del vector
#dim: 0 o 1, 0 para filas y 1 para columnas
def omeg(matriz, vector, dim):
    counter = 0
    if dim == 0:
        matriz = matriz[vector:] + matriz
        for i in range(len(matriz)):
            if(zero_vector(matriz[i+1])):
                counter += 1
            else:
                break
    if dim == 1:
        t_matriz = transpose_matrix(matriz)
        t_matriz = t_matriz[vector:] + t_matriz
        for i in range(len(t_matriz)):
            if(zero_vector(t_matriz[i+1])):
                counter += 1
            else:
                break
    return counter

#"conjuga" la matriz, i.e., cambia las entradas 1 por 0 y viceversa
def conjugate_matrix(matriz):
    m = matriz
    for i in m:
        for j in range(len(i)):
            i[j] = 1 if (i[j]==0) else 0
    return m

#calcula la distancia aparente de una hipercolumna
#hm: lista de enteros, la hipercolumna
def dist_Hm(hm):
    om = []
    indices = []
    if zero_vector(hm):
        return 0
    for i in range(0,len(hm)):
        if hm[i] != 0:
            indices.append(i)
    for i in indices:
        om.append(omeg(hm,i,0)+1)
    return max(om)  

#genera el conjunto de ds-bounds con la distancia aparente
#words: lista de listas, conjunto de vectores binarios
def generate_ad(words):
    dsb_set = []
    for word in words:
        b = dist_Hm(word)
        s = defining_set(word)
        dsb_set.append([s,b])
    return dsb_set

#genera el conjunto de b's a probar en la cota HT
def generate_b(delta):
    return [x for x in range(1, delta + 1)]

#genera los s+1 multiplos de b modulo n
def multiples(b, s, n):
    m = []
    for i in range(s+1):
        m.append((b*i)%n)
    return m

#hace la suma 'distributiva' modulo n de conjuntos usada en la cota HT
def set_sum(A, B, n):
    AplusB = [] 
    for i in A:
        for j in B:
            AplusB.append((i+j)%n)
    return AplusB

#trae el indice del primer elemento no nulo del vector
def first_one(vector):
    for i in range(len(vector)):
            if vector[i] == 1:
                return i
    return

#genera el conjunto de definicion del vector
def defining_set(vector):
    return [i for i, e in enumerate(vector) if e == 0]

#separa los indices de los 'bloques' de ceros consecutivos del vector 
def zero_blocks(t):
    first = first_one(t)
    if zero_vector(t):
        return []
    T = t + t
    bloques = []
    bloque = []
    for i in range(first,len(T)):
        if T[i] == 0:
            bloque.append(i%len(t))
        else:
            if bloque in bloques:
                break
            if len(bloque) > 0:
                bloques.append(bloque)
                bloque = []
    return bloques

#calcula la cota HT del vector
def HT(vector):
    if zero_vector(vector):
        return 0
    n = len(vector)
    cotas = []
    T = defining_set(vector)
    blocks = zero_blocks(vector)
    max_delta = (max([len(x) for x in blocks]) + 1) if blocks else 1
    for block in blocks:
        delta = len(block) + 1
        bs = generate_b(n)
        bs = [b for b in bs if gcd(b,n) < delta]
        ss = [s for s in range(1, delta*4)]
        for b in bs:
            for s in ss:
                B = multiples(b,s,n)
                AB = set_sum(block,B,n)
                if set(AB).issubset(T):
                    cotas.append(delta+s)
    return max(cotas) if cotas else max_delta

#genera el conjunto de ds-bounds con la distancia aparente fuerte
#words: lista de listas, conjunto de vectores binarios
def generate_ht(words):
    dsb_set = []
    for word in words:
        b = HT(word)
        s = defining_set(word)
        dsb_set.append([s,b])
    return dsb_set

In [2]:
#retorna la ds-bound optima para un conjunto de indices N con relacion a un conjunto de ds-bounds definido
def optimal_dsbound(ds_bound, N):
    maximo = 0
    for bound in ds_bound:
        if bound[0] == N:
            maximo = bound[1] if bound[1] > maximo else maximo
    return maximo

In [3]:
#calcula la distancia delta-aparente de un vector
def dad_vector(ds_bound, vector):
    if zero_vector(vector):
        return 0
    supp = vector_support(vector)
    complement = []
    for i in range(len(vector)):
        if i not in supp:
            complement.append(i)
    optimal = optimal_dsbound(ds_bound, complement)
    return optimal

In [4]:
#calcula la distancia B-aparente de un vector a partir de un conjunto de ds-bounds
def Bad_vector(dsb_set, vector):
    if zero_vector(vector):
        return 0
    maximo = 0
    for dsbound in dsb_set:
        dadv = dad_vector(dsbound, vector)
        maximo = dadv if dadv > maximo else maximo
    return maximo

In [5]:
#calcula el soporte en filas/columnas para la matriz M
def support(M):
    supp = []
    for index in range(len(M)):
        if not zero_vector(M[index]):
            supp.append(index)
    return supp

In [6]:
#calcula el omega en las filas de la matriz M con respecto a un conjunto de ds-bounds
def omega_x(M, dsb_set):
    soporte = support(M)
    complemento = []
    omega = 0
    for i in range(len(M)):
        if i not in soporte:
            complemento.append(i)
    for dsbound in dsb_set:
        opt = optimal_dsbound(dsbound, complemento)
        omega = opt if opt > omega else omega
    print("Omega respecto a x: ", omega)
    return omega

#calcula el epsilon en las filas de la matriz M con respecto a un conjunto de ds-bounds
def epsilon_x(M, dsb_set):
    soporte = support(M)
    epsilon = 0
    for i in soporte:
        bv = Bad_vector(dsb_set, M[i])
        epsilon = bv if bv > epsilon else epsilon
    print("Epsilon respecto a x: ", epsilon)
    return epsilon

#calcula la distancia B-aparente en las filas de la matriz M con respecto a los dos conjuntos de 
#ds-bounds (uno para cada dimension r1 y r2 de la matriz M_(r1 x r2))
def Bad_x(dsb_set, dsb_set_2, M):
    print("------------------------------------------------------------------------")
    ome = omega_x(M, dsb_set)
    eps = epsilon_x(M, dsb_set_2)
    print("---------------------------------------------------------------")
    print("Distancia B-aparente respecto a x: ", ome*eps)
    print("------------------------------------------------------------------------")
    return ome*eps

In [7]:
#calcula el omega en las columnas de la matriz M con respecto a un conjunto de ds-bounds
def omega_y(m, dsb_set):
    M = transpose_matrix(m)
    soporte = support(M)
    complemento = []
    omega = 0
    for i in range(len(M)):
        if i not in soporte:
            complemento.append(i)
    for dsbound in dsb_set:
        opt = optimal_dsbound(dsbound, complemento)
        omega = opt if opt > omega else omega
    print("Omega respecto a y: ", omega)
    return omega 

#calcula el epsilon en las columnas de la matriz M con respecto a un conjunto de ds-bounds
def epsilon_y(m, dsb_set):
    M = transpose_matrix(m)
    soporte = support(M)
    epsilon = 0
    for i in soporte:
        bv = Bad_vector(dsb_set, M[i])
        epsilon = bv if bv > epsilon else epsilon
    print("Epsilon respecto a y: ", epsilon)
    return epsilon

#calcula la distancia B-aparente en las columnas de la matriz M con respecto a los dos conjuntos de 
#ds-bounds (uno para cada dimension r1 y r2 de la matriz M_(r1 x r2))
def Bad_y(dsb_set, dsb_set_2, M):
    ome = omega_y(M, dsb_set)
    eps = epsilon_y(M, dsb_set_2)
    print("---------------------------------------------------------------")
    print("Distancia B-aparente respecto a y: ", ome*eps)
    print("------------------------------------------------------------------------")
    return ome*eps

In [8]:
#calcula la distancia B-aparente de la matriz M con respecto a los dos conjuntos de 
#ds-bounds (uno para cada dimension r1 y r2 de la matriz M_(r1 x r2))
def Bad_matrix(M,dsb_set,dsb_set_2):
    bad = max(Bad_x(dsb_set,dsb_set_2, M), Bad_y(dsb_set_2,dsb_set, M))
    return bad

Generar los set de ds-bounds para cada Z_r

In [9]:
palabras_15 = generate_binary(15)
palabras_3 = generate_binary(3)
palabras_7 = generate_binary(7)
palabras_5 = generate_binary(5)
palabras_9 = generate_binary(9)

In [10]:
ad_15 = generate_ad(palabras_15)
ht_15 = generate_ht(palabras_15)
ad_5 = generate_ad(palabras_5)
ht_5 = generate_ht(palabras_5)
ad_7 = generate_ad(palabras_7)
ht_7 = generate_ht(palabras_7)
ad_3 = generate_ad(palabras_3)
ht_3 = generate_ht(palabras_3)
ad_9 = generate_ad(palabras_9)
ht_9 = generate_ht(palabras_9)

In [11]:
dsb_set_15 = [ad_15,ht_15]
dsb_set_3 = [ad_3,ht_3]
dsb_set_7 = [ad_7,ht_7]
dsb_set_5 = [ad_5,ht_5]
dsb_set_9 = [ad_9,ht_9]

<b>Ejemplo 1</b>

In [12]:
matrix =  generate_matrix(2,[[0,0],[0,1],[0,3],[1,0],[1,1]],[5,15])
pm(matrix)

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [13]:
Bad_matrix(matrix,dsb_set_5,dsb_set_15)

------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  6
---------------------------------------------------------------
Distancia B-aparente respecto a x:  6
------------------------------------------------------------------------
Omega respecto a y:  2
Epsilon respecto a y:  3
---------------------------------------------------------------
Distancia B-aparente respecto a y:  6
------------------------------------------------------------------------


6

<b>Ejemplo 2</b>

In [14]:
matrix = generate_matrix(2,[[0,0],[0,3],[0,5],[0,7],[1,0],[1,2],[1,4]],[5,15])
pm(matrix)

[0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]


Calcular la distancia B-aparente de la matriz

In [15]:
Bad_matrix(matrix,dsb_set_5,dsb_set_15)

------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  8
---------------------------------------------------------------
Distancia B-aparente respecto a x:  8
------------------------------------------------------------------------
Omega respecto a y:  2
Epsilon respecto a y:  3
---------------------------------------------------------------
Distancia B-aparente respecto a y:  6
------------------------------------------------------------------------


8

<b>Ejemplo 3</b>

In [16]:
matrix = generate_matrix(2,[[0,3],[0,5],[1,1]],[5,15])
pm(matrix)
Bad_matrix(matrix,dsb_set_5,dsb_set_15)

[1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  4
---------------------------------------------------------------
Distancia B-aparente respecto a x:  4
------------------------------------------------------------------------
Omega respecto a y:  1
Epsilon respecto a y:  2
---------------------------------------------------------------
Distancia B-aparente respecto a y:  2
------------------------------------------------------------------------


4

<b>Ejemplo 4</b>

In [17]:
matrix = conjugate_matrix(generate_matrix(2,[[0,0],[1,0],[1,3]],[5,7]))
pm(matrix)

[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]


In [18]:
Bad_matrix(matrix,dsb_set_5,dsb_set_7)

------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  7
---------------------------------------------------------------
Distancia B-aparente respecto a x:  7
------------------------------------------------------------------------
Omega respecto a y:  3
Epsilon respecto a y:  2
---------------------------------------------------------------
Distancia B-aparente respecto a y:  6
------------------------------------------------------------------------


7

<b>Ejemplo 5</b>

In [19]:
matrix = conjugate_matrix(generate_matrix(2,[[1,0],[1,3]],[5,7]))
pm(matrix)

[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]


In [20]:
Bad_matrix(matrix,dsb_set_5,dsb_set_7)

------------------------------------------------------------------------
Omega respecto a x:  2
Epsilon respecto a x:  3
---------------------------------------------------------------
Distancia B-aparente respecto a x:  6
------------------------------------------------------------------------
Omega respecto a y:  3
Epsilon respecto a y:  2
---------------------------------------------------------------
Distancia B-aparente respecto a y:  6
------------------------------------------------------------------------


6

<b>Ejemplo 6</b>

In [21]:
matrix = conjugate_matrix(generate_matrix(2,[[1,0],[2,5],[2,7]],[3,15]))
pm(matrix)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]


In [22]:
Bad_matrix(matrix,dsb_set_3,dsb_set_15)

------------------------------------------------------------------------
Omega respecto a x:  2
Epsilon respecto a x:  10
---------------------------------------------------------------
Distancia B-aparente respecto a x:  20
------------------------------------------------------------------------
Omega respecto a y:  5
Epsilon respecto a y:  3
---------------------------------------------------------------
Distancia B-aparente respecto a y:  15
------------------------------------------------------------------------


20

<b>Ejemplo 7 (Diana)</b>

In [23]:
matrix = generate_matrix(2,[[0,0],[1,0],[0,3]],[5,7])
pm(matrix)

[0, 1, 1, 0, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1]


In [24]:
Bad_matrix(matrix,dsb_set_5,dsb_set_7)

------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  4
---------------------------------------------------------------
Distancia B-aparente respecto a x:  4
------------------------------------------------------------------------
Omega respecto a y:  2
Epsilon respecto a y:  2
---------------------------------------------------------------
Distancia B-aparente respecto a y:  4
------------------------------------------------------------------------


4

<b>Ejemplo 8 (Diana)</b>

In [25]:
matrix = generate_matrix(2,[[0,1],[0,3],[1,3]],[5,7])
pm(matrix)

[1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 1, 0, 0]
[1, 1, 1, 0, 1, 0, 0]
[1, 1, 1, 0, 1, 0, 0]
[1, 1, 1, 0, 1, 0, 0]


In [26]:
Bad_matrix(matrix,dsb_set_5,dsb_set_7)

------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  7
---------------------------------------------------------------
Distancia B-aparente respecto a x:  7
------------------------------------------------------------------------
Omega respecto a y:  3
Epsilon respecto a y:  2
---------------------------------------------------------------
Distancia B-aparente respecto a y:  6
------------------------------------------------------------------------


7

<b>Ejemplo 9 (Diana)</b>

In [27]:
matrix = generate_matrix(2,[[0,1],[1,0]],[5,9])
pm(matrix)

[1, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1]


In [28]:
Bad_matrix(matrix,dsb_set_5,dsb_set_9)

------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  3
---------------------------------------------------------------
Distancia B-aparente respecto a x:  3
------------------------------------------------------------------------
Omega respecto a y:  1
Epsilon respecto a y:  5
---------------------------------------------------------------
Distancia B-aparente respecto a y:  5
------------------------------------------------------------------------


5

<b>Ejemplo 10 (Diana)</b>

In [29]:
matrix = generate_matrix(2,[[0,0],[1,0],[0,7]],[3,15])
pm(matrix)

[0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [30]:
Bad_matrix(matrix,dsb_set_3,dsb_set_15)

------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  4
---------------------------------------------------------------
Distancia B-aparente respecto a x:  4
------------------------------------------------------------------------
Omega respecto a y:  2
Epsilon respecto a y:  2
---------------------------------------------------------------
Distancia B-aparente respecto a y:  4
------------------------------------------------------------------------


4

In [31]:
matrix = generate_matrix(2,[[1,0],[0,1],[1,3],[1,6]],[3,9])
pm(matrix)

[1, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 1, 1, 0, 1, 1, 0, 1, 1]
[0, 1, 1, 0, 1, 1, 0, 1, 1]


In [32]:
Bad_matrix(matrix,dsb_set_3,dsb_set_9)

------------------------------------------------------------------------
Omega respecto a x:  1
Epsilon respecto a x:  3
---------------------------------------------------------------
Distancia B-aparente respecto a x:  3
------------------------------------------------------------------------
Omega respecto a y:  1
Epsilon respecto a y:  3
---------------------------------------------------------------
Distancia B-aparente respecto a y:  3
------------------------------------------------------------------------


3

In [33]:
q_orbits(2, [0,0], [5,15])

[[0, 0]]

In [34]:
q_orbits(2, [0,1], [5,15])

[[0, 1], [0, 2], [0, 4], [0, 8]]

In [35]:
q_orbits(2, [0,3], [5,15])

[[0, 3], [0, 6], [0, 12], [0, 9]]

In [36]:
q_orbits(2, [1,0], [5,15])

[[1, 0], [2, 0], [4, 0], [3, 0]]

In [37]:
q_orbits(2, [1,1], [5,15])

[[1, 1], [2, 2], [4, 4], [3, 8]]